In [ ]:
import pandas as pd
from src.prediction import config
from loguru import logger
data = pd.read_parquet(r"data\parquet\predicted_votes.parquet")
def vote_counts_to_result(votes: pd.Series) -> str:
    """Convert vote counts to a result string."""
    max_votes = votes[["yes", "no", "abstain"]].idxmax()
    if max_votes == "yes":
        return "zustimmung"
    elif max_votes == "no":
        return "ablehnung"
    elif max_votes == "abstain":
        return "enthaltung"
    else:
        logger.warning(f"Unexpected vote counts: {votes}")
        return None

# TODO flip around for beschlussempfelungen that vote against the recommendation
def mirror_decision(decision: str) -> str:
    """Mirror the decision for beschlussempfehlungen."""
    if decision == "zustimmung":
        return "ablehnung"
    elif decision == "ablehnung":
        return "zustimmung"
    else:
        return decision

for party in config.PARTIES:
    real_votes = pd.read_csv(f"data/csv/vote_counts/{party}.csv").rename(
        columns={"vote_id": "vote"}
    ).drop_duplicates(subset="vote")
    real_votes[f"{party}_ground_truth"] = real_votes.apply(vote_counts_to_result, axis=1)
    data = data.merge(real_votes[["vote", f"{party}_ground_truth"]], on="vote", how="left")
    data[f"{party}_prediction"] = data[f"{party}_decision"].str["decision"]
    # Flip the decision for beschlussempfehlungen if they are against the underlying decision
    # This is necessary because the beschlussempfehlung is a recommendation to vote against, thus changing the context
    beschlussempfehlungen_filt = (data["beschlussempfehlung"] == "ablehnen")
    data.loc[beschlussempfehlungen_filt, f"{party}_prediction"] = data.loc[
        beschlussempfehlungen_filt, f"{party}_prediction"
    ].apply(mirror_decision)

In [14]:
data[(data["beschlussempfehlung"] == "ablehnen")]

,vote,drucksache_id,beschlussempfehlung,title,type,content,summary,embedding,date,AfD_decision,...,DIE_GRÜNEN_prediction,DIE_LINKE_ground_truth,DIE_LINKE_prediction,FDP_ground_truth,FDP_prediction,SPD_ground_truth,SPD_prediction,Union_ground_truth,Union_prediction,category
8,20240613_2,20/11372,ablehnen,"Antrag der Abgeordneten Dr. Bernd Baumann, Dr....",Antrag,"Antrag der Abgeordneten Dr. Bernd Baumann, Dr....",Der Bundestag soll über ein Verbot des Vereins...,"[-0.03824455663561821, 0.018761679530143738, 0...",2024-06-13,{'context': '- Islam: - Wirken von Islam und...,...,ablehnung,zustimmung,ablehnung,zustimmung,ablehnung,zustimmung,ablehnung,zustimmung,ablehnung,"Inneres & Migration - Innere Sicherheit, öffen..."
9,20240606_1,20/11393,ablehnen,Antrag der Fraktion der CDU/CSU Den politische...,Antrag,Antrag der Fraktion der CDU/CSU Den politische...,Der Bundestag soll über Maßnahmen zur Bekämpfu...,"[-0.026766983792185783, 0.02231553941965103, 0...",2024-06-06,{'context': '- Islam: - Wirken von Islam und...,...,ablehnung,zustimmung,zustimmung,zustimmung,ablehnung,zustimmung,ablehnung,ablehnung,ablehnung,"Inneres & Migration - Innere Sicherheit, öffen..."
10,20240605_1,20/11149,ablehnen,"Antrag der Abgeordneten Kay Gottschalk, Klaus ...",Antrag,"Antrag der Abgeordneten Kay Gottschalk, Klaus ...",Es wird die vollständige Abschaffung des Solid...,"[0.02055482566356659, 0.006369335111230612, 0....",2024-06-05,{'context': '- Steuern und Finanzen: - Keine...,...,zustimmung,zustimmung,zustimmung,zustimmung,ablehnung,zustimmung,zustimmung,zustimmung,ablehnung,"Finanzen - Steuern, Staatsbudget, Haushalts- u..."
15,20240118_2,20/5551,ablehnen,"Antrag der Abgeordneten Dr. Alexander Gauland,...",Antrag,"Antrag der Abgeordneten Dr. Alexander Gauland,...",Der Bundestag soll über eine Friedensinitiativ...,"[-0.05222005769610405, 0.017403464764356613, -...",2024-01-18,{'context': '- Außen- und Verteidigungspolitik...,...,ablehnung,NaN,ablehnung,zustimmung,zustimmung,zustimmung,ablehnung,zustimmung,zustimmung,"Verteidigung & Sicherheit - Militär, Verteidig..."
16,20231130_1,20/7295,ablehnen,"Antrag der Abgeordneten Christian Görke, Dr. G...",Antrag,"Antrag der Abgeordneten Christian Görke, Dr. G...",Der Bundestag stimmt über einen Antrag zur Ref...,"[0.05590945854783058, 0.010546945035457611, 0....",2023-11-30,{'context': '- Steuern und Finanzen: - Keine...,...,ablehnung,ablehnung,ablehnung,zustimmung,zustimmung,zustimmung,ablehnung,zustimmung,zustimmung,"Finanzen - Steuern, Staatsbudget, Haushalts- u..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
206,20140130_1,18/180,ablehnen,"Antrag der Abgeordneten Harald Ebner, Bärbel H...",Antrag,Deutscher Bundestag\nDrucksache 18/180 18. Wah...,Es wird über die Ablehnung der Zulassung der g...,"[-0.015334045514464378, 0.04127142205834389, 0...",2014-01-30,{'context': '- Unterstützung eines Europas sou...,...,ablehnung,ablehnung,ablehnung,NaN,zustimmung,zustimmung,ablehnung,zustimmung,ablehnung,"Landwirtschaft & Ernährung - Agrarpolitik, Ern..."
211,20130606,17/12485,ablehnen,"Antrag der Abgeordneten Michael Groß, Sören Ba...",Antrag,Deutscher Bundestag Drucksache 17/12485 17. Wa...,Der Antrag fordert Maßnahmen für bezahlbares W...,"[0.007606168743222952, 0.0040185037069022655, ...",2013-06-06,{'context': '- Nachhaltiges Energiekonzept für...,...,ablehnung,enthaltung,ablehnung,zustimmung,ablehnung,ablehnung,ablehnung,zustimmung,ablehnung,"Wohnen & Stadtentwicklung - Wohnungsbau, Städt..."
212,20130606,17/12481,ablehnen,"Antrag der Abgeordneten Heidrun Bluhm, Halina ...",Antrag,Deutscher Bundestag Drucksache 17/12481 17. Wa...,Der Antrag fordert Maßnahmen zur Bekämpfung de...,"[0.005351749248802662, 0.021321283653378487, 0...",2013-06-06,{'context': '- Nachhaltiges Energiekonzept für...,...,ablehnung,enthaltung,ablehnung,zustimmung,zustimmung,ablehnung,ablehnung,zustimmung,ablehnung,"Wohnen & Stadtentwicklung - Wohnungsbau, Städt..."
213,20130606,17/11696,ab

In [12]:
data

,vote,drucksache_id,beschlussempfehlung,title,type,content,summary,embedding,date,AfD_decision,...,DIE_GRÜNEN_prediction,DIE_LINKE_ground_truth,DIE_LINKE_prediction,FDP_ground_truth,FDP_prediction,SPD_ground_truth,SPD_prediction,Union_ground_truth,Union_prediction,category
0,20250130_4,20/14047,annehmen,Antrag der Bundesregierung Fortsetzung der Bet...,Antrag,Antrag der Bundesregierung Fortsetzung der Bet...,Es wird über die Fortsetzung der deutschen Bet...,"[-0.032250940799713135, 0.04619118571281433, 0...",2025-01-30,{'context': '- Außen- und Verteidigungspolitik...,...,zustimmung,ablehnung,ablehnung,zustimmung,zustimmung,zustimmung,zustimmung,zustimmung,zustimmung,"Verteidigung & Sicherheit - Militär, Verteidig..."
1,20250130_3,20/14046,annehmen,Antrag der Bundesregierung Fortsetzung der Bet...,Antrag,Antrag der Bundesregierung Fortsetzung der Bet...,Der Bundestag stimmt über die Fortsetzung der ...,"[-0.023844607174396515, 0.07453092187643051, 0...",2025-01-30,{'context': '- Außen- und Verteidigungspolitik...,...,zustimmung,ablehnung,ablehnung,zustimmung,zustimmung,zustimmung,zustimmung,zustimmung,zustimmung,"Verteidigung & Sicherheit - Militär, Verteidig..."
2,20250130_2,20/14045,annehmen,Antrag der Bundesregierung Fortsetzung der Bet...,Antrag,Antrag der Bundesregierung Fortsetzung der Bet...,Der Bundestag stimmt über die Fortsetzung der ...,"[-0.026726506650447845, 0.04273178055882454, 0...",2025-01-30,{'context': '- Außen- und Verteidigungspolitik...,...,zustimmung,ablehnung,ablehnung,zustimmung,zustimmung,zustimmung,zustimmung,zustimmung,zustimmung,"Verteidigung & Sicherheit - Militär, Verteidig..."
3,20250130_1,20/14044,annehmen,Antrag der Bundesregierung Fortsetzung der Bet...,Antrag,Antrag der Bundesregierung Fortsetzung der Bet...,Der Bundestag stimmt über die Fortsetzung der ...,"[-0.03594012185931206, 0.025598490610718727, 0...",2025-01-30,{'context': '- Außen- und Verteidigungspolitik...,...,zustimmung,ablehnung,ablehnung,zustimmung,zustimmung,zustimmung,zustimmung,zustimmung,zustimmung,"Verteidigung & Sicherheit - Militär, Verteidig..."
4,20241017_3,20/12893,annehmen,Antrag der Bundesregierung Fortsetzung des Ein...,Antrag,Antrag der Bundesregierung Fortsetzung des Ein...,Der Bundestag stimmt über die Fortsetzung des ...,"[-0.0020443603862076998, 0.058881428092718124,...",2024-10-17,{'context': '- Außen- und Verteidigungspolitik...,...,zustimmung,ablehnung,ablehnung,zustimmung,zustimmung,zustimmung,zustimmung,zustimmung,zustimmung,"Verteidigung & Sicherheit - Militär, Verteidig..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
618,20121025_5,17/11196,None,Änderungsantrag der Abgeordneten Volker Beck (...,Änderungsantrag,Deutscher Bundestag Drucksache 17/11196 17. Wa...,Der Antrag fordert die steuerrechtliche Gleich...,"[0.027212975546717644, 0.02205045148730278, 0....",2012-10-25,None,...,zustimmung,zustimmung,zustimmung,ablehnung,zustimmung,zustimmung,zustimmung,ablehnung,zustimmung,"Arbeit & Soziales - Arbeitsmarktpolitik, Sozia..."
619,20121025_4,17/11193,None,Änderungsantrag der Fraktion der SPD zu der zw...,Änderungsantrag,Deutscher Bundestag Drucksache 17/11193 17. Wa...,Es wird die Abschaffung des ermäßigten Umsatzs...,"[-0.007618354167789221, 0.011488755233585835, ...",2012-10-25,None,...,ablehnung,zustimmung,ablehnung,ablehnung,ablehnung,zustimmung,ablehnung,ablehnung,ablehnung,"Finanzen - Steuern, Staatsbudget, Haushalts- u..."
620,20121025_3,17/11172,None,Änderungsantrag der Abgeordneten Dr. Gerhard S...,Änderungsantrag,Deutscher Bundestag Drucksache 17/11172 17. Wa...,Es wird über eine Änderung des Wertpapierhande...,"[-0.010517100803554058, 0.019554799422621727, ...",2012-10-25,None,...,zustimmung,zustimmung,zustimmung,ablehnung,zustimmung,enthaltung,zustimmung,ablehnung,zustimmung,"Justiz & Verbraucherschutz - Rechtsprechung, G..."
621,20121025_2,17/10059,None,Gesetzentwurf der Bundesregierung Entwurf eine...,Gesetzentwurf,A. Problem und Ziel Die intensiv

In [3]:
from src.utils.openai_utils import get_embedding
import os

if os.path.exists("data/parquet/kategorien.parquet"):
    categories = pd.read_parquet("data/parquet/kategorien.parquet")
else:
    categories = pd.DataFrame({
        "kategorie": [
        "Finanzen - Steuern, Staatsbudget, Haushalts- und Finanzpolitik",
        "Inneres & Migration - Innere Sicherheit, öffentliche Verwaltung, Migration, Staatsbürgerschaft",
        "Außenpolitik & Europäische Angelegenheiten - Diplomatie, internationale Beziehungen, EU-Politik",
        "Verteidigung & Sicherheit - Militär, Verteidigungsstrategie, Bundeswehr, Rüstung",
        "Wirtschaft & Energie - Industriepolitik, Mittelstand, Energieversorgung, Wirtschaftsordnungen",
        "Forschung & Technologie - Innovationsförderung, Raumfahrt, Forschungseinrichtungen, Technologietransfer",
        "Justiz & Verbraucherschutz - Rechtsprechung, Gesetzgebung, Verbraucherschutz, Datenschutz",
        "Bildung, Familie & Jugend - Schulen, Hochschulen, Familienförderung, Kinder- und Jugendpolitik",
        "Arbeit & Soziales - Arbeitsmarktpolitik, Sozialversicherung, Renten, Integration",
        "Digitalisierung & Modernisierung - E-Government, IT-Infrastruktur, digitale Verwaltung, Cybersecurity",
        "Verkehr & Infrastruktur - Straßen-, Schienen- und Luftverkehr, Mobilitätskonzepte, Infrastrukturprojekte",
        "Umwelt, Klima & Naturschutz - Umweltschutz, Klimapläne, Artenschutz, nukleare Sicherheit",
        "Gesundheit - Gesundheitssystem, Krankenversicherung, Arzneimittelregulierung, Pandemie- und Präventionspolitik",
        "Landwirtschaft & Ernährung - Agrarpolitik, Ernährungssicherheit, Ländliche Entwicklung",
        "Entwicklungszusammenarbeit - Entwicklungsprojekte, humanitäre Hilfe, internationale Zusammenarbeit",
        "Wohnen & Stadtentwicklung - Wohnungsbau, Städtebau, Bauordnung, Städtebauförderung"
    ]})
    categories["embedding"] = categories["kategorie"].apply(get_embedding)
    categories.to_parquet("data/parquet/kategorien.parquet", index=False)

In [4]:
import numpy as np
def get_closest_category(summary_embedding: np.array) -> str:
    distances = categories["embedding"].apply(
        lambda x: np.linalg.norm(np.array(x) - summary_embedding)
    )
    closest_index = distances.idxmin()
    return categories.iloc[closest_index]["kategorie"]


data["category"] = data["embedding"].apply(get_closest_category)

In [5]:
percentage_partyline = {}

for party in config.PARTIES:
    party_votes = data[data[f"{party}_ground_truth"].notna()]
    partyline_correct = (party_votes[f"{party}_ground_truth"] == party_votes[f"{party}_prediction"]).sum() / len(party_votes)
    percentage_partyline[party] = round(partyline_correct * 100)

In [6]:
import plotly.graph_objects as go

percentage_partyline = {
    'AfD': 55,
    'DIE_GRÜNEN': 58,
    'DIE_LINKE': 59,
    'FDP': 65,
    'SPD': 70,
    'Union': 67
}

colors = {
    'AfD': '#009EE0',
    'DIE_GRÜNEN': '#00A65A',
    'DIE_LINKE': '#BE3075',
    'FDP': '#FFD700',
    'SPD': '#E3000F',
    'Union': '#000000'
}

fig = go.Figure(go.Bar(
    x=list(percentage_partyline.keys()),
    y=list(percentage_partyline.values()),
    marker_color=[colors[party] for party in percentage_partyline],
    width=0.4
))

fig.update_layout(
    title='Percentage of Party-Line Agreement',
    xaxis_title='Political Party',
    yaxis_title='Agreement (%)',
    yaxis=dict(range=[0, 100]),
    template='plotly_dark'
)

fig.update_traces(text=list(percentage_partyline.values()), textposition='auto')

fig.show()


In [7]:
percentage_partyline_by_category = {}

for party in config.PARTIES:
    party_votes = data[data[f"{party}_ground_truth"].notna()].copy()
    party_votes["partyline_correct"] = (party_votes[f"{party}_ground_truth"] == party_votes[f"{party}_prediction"])
    categories_vc = party_votes["category"].value_counts()
    relevant_categories = categories_vc[categories_vc > 20].index.tolist()
    party_votes = party_votes[party_votes["category"].isin(relevant_categories)]
    party_votes.groupby("category")["partyline_correct"].mean().reset_index()
    percentage_partyline_by_category[party] = party_votes.groupby("category")["partyline_correct"].mean().reset_index().to_dict(orient="records")

In [8]:
import pandas as pd
import plotly.express as px

# build a DataFrame: rows = parties, columns = categories
df = pd.DataFrame({
    party: {item['category']: item['partyline_correct'] for item in entries}
    for party, entries in percentage_partyline_by_category.items()
}).T

# reshape to long form for px.bar
df_long = df.reset_index().melt(
    id_vars='index',
    var_name='Category',
    value_name='Correctness'
)
df_long.rename(columns={'index': 'Party'}, inplace=True)

fig = px.bar(
    df_long,
    x='Party',
    y='Correctness',
    color='Category',
    barmode='group'
)

fig.update_layout(
    xaxis_title='Party',
    yaxis_title='Partyline Correctness',
    legend_title_text='Category'
)

fig.show()


In [9]:
data.iloc[620]

vote                                                              20121025_3
drucksache_id                                                       17/11172
beschlussempfehlung                                                     None
title                      Änderungsantrag der Abgeordneten Dr. Gerhard S...
type                                                         Änderungsantrag
content                    Deutscher Bundestag Drucksache 17/11172 17. Wa...
summary                    Es wird über eine Änderung des Wertpapierhande...
embedding                  [-0.010517100803554058, 0.019554799422621727, ...
date                                                     2012-10-25 00:00:00
AfD_decision                                                            None
DIE_GRÜNEN_decision        {'context': '- Verbraucherschutz und Verbrauch...
DIE_LINKE_decision         {'context': '- Schutzschirm für Menschen: Scha...
FDP_decision               {'context': '- Stärkung des Mittelstandes durc...